# Trabajo de Curso VC

### Autores
Kilian Armas Pérez

Pablo Segura López

In [ ]:
# pip install torch==1.10.0+cpu torchvision==0.11.0+cpu torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
# pip install matplotlib
# pip install numpy
# pip install ffmpeg
# pip install opencv-python
# pip install scipy
# pip install torchsummary
# añadir el checkpoint
# pip install ffprobe
# Añadir al path el ffmpeg, la carpeta bin